#### Mating data process pipeline

In [7]:
import sys
sys.path.append("../")
import os

import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange

from cellmating.utils import file_traverse, load_from_pkl
from cellmating.io import imread

from script_wlli.mating_feature_extract import extract_mating_features, stack_features

In [8]:
root = "/media/wlli/T7/data/mating_all/"
file_list = file_traverse(root, file_regular=r".*/selected/.*.tif")
print("totle images: ", len(file_list))

totle images:  195


In [9]:
from script_wlli.utils import load_configs
config = load_configs("./config_mating.yml")

In [10]:
all_images_feature_over_time = {}
for i in trange(0, len(file_list)):
    file_name = file_list[i]
    try:
        image = imread(file_name)
        g = load_from_pkl(file_name[:-4]+".network")
        key = int(file_name.split("_")[-1][:-4])
        key = np.unique(image[-1,-1][image[-1, -2] == key])[0]%1000
        # extract features
        data = extract_mating_features(image, g, key)
        # stack features
        all_images_feature_over_time = stack_features(all_images_feature_over_time, data, os.path.basename(file_name), config=config)
    except Exception as error:
    # handle the exception
        print(file_name," : ", error)

100%|███████████████████████████████████████| 195/195 [1:09:01<00:00, 21.24s/it]


In [12]:
from cellmating.utils import dump_to_pkl

In [63]:
dump_to_pkl(all_images_feature_over_time,"./data/extract_feature_object/all_image_time_feature_20240408.obj")
for key in all_images_feature_over_time:
    data = all_images_feature_over_time[key]
    data.to_csv("./data/time_split_csv/20240408/"+key+".csv")

In [76]:
data = all_images_feature_over_time["-7"].iloc[0]
if data.p ==data.ref:
    print(data.image,
         "\nP: ",data.p,
         "\ntime:", data.time_measure,
         "\n#nei_p: ", data.neighbor_p,
         "\n#nei_m: ", data.neighbor_m,)

20220802_YAV2104xYAV2105_001_01_R3D_REF_rigid_003005.tif 
P:  57 
time: 47 
#nei_p:  6 
#nei_m:  2


In [71]:
data.p

24

In [72]:
data.m

16